In [1]:
#IMPORT DAS BIBLIOTECAS NECESSÁRIAS
import numpy as np #PARA DEFINIR OS PONTOS DE OBSERVAÇÕES 
from GeneticAlgorithm import GeneticAlgorithm #IMPORTAR A CLASSE ALGORITMO GENETICO
import matplotlib.pyplot as plt #PARA VISUALIZAR O RESULTADO
from Models import rect #IMPORTANDO O MODELO
from Data.Functionals import phi #IMPORTANDO A FUNCAO DE AJUSTE
from Data.Population import Fontes #PRA CRIAR A POPULACAO USANDO O MELHOR DEPOIS DO GENETICO

In [2]:
#DEFINICAO DOS PARAMETROS PRO GENETICO
xobs = np.linspace(-1000,1000,500) #SO PRA CALCULAR O GZ DO MELHOR INDIVIDUO DEPOIS DO GENETICO
zobs = np.zeros( len( xobs ) ) #SO PRA CALCULAR O GZ DO MELHOR INDIVIDUO DEPOIS DO GENETICO
model = rect( -100,100,300,900,2 ) #CRIANDO O MODELO
model_gz = model.Gz( xobs, zobs ) #CALCULANDO O GZ DO MODELO
model_gz_noised = model.addnoise( ) #CALCULANDO O GZ DO MODELO + GAUSSIAN NOISE
min_bounds = [ -1000, 200, 1e5 ] #ONDE PROCURAR AS SOLUCOES
max_bounds = [ 1000, 800, 1e10 ] #ONDE PROCURAR AS SOLUCOES
nfontes = 20 #NUMERO DE BOLINHAS DE CADA INDIVIDUO
nind = 5 #NUMERO DE INDIVIDUOS
pmut = 0.1 #PROBABILIDADE DE CADA INDIVIDUO MUTAR
mu = 0.4 #FATOR DE REGULARIZACAO

In [ ]:
#ALGORITMO GENETICO
ga = GeneticAlgorithm( phi, [ model_gz_noised, 'L2', mu ], min_bounds, max_bounds, pmut, nfontes, nind) #CRIANDO O AG USANDO OS PARAMETROS ESCOLHIDOS
ga.start( 500 ) #COMECANDO AS ITERACOES (NESTE CASO, 500)

pop = Fontes( ) #CRIANDO UMA NOVA POPULACAO PRA GUARDAR O MELHOR DO GENETICO
best = pop.Gera_from_Existing( [ ga.winner ] ) #GUARDANDO O MELHOR DO GENETICO NESSA NOVA POPULACAO(QUE É UMA CLASSE) ( O MELHOR DO GENETICO TEM O NOME DE ga.winner)
gz_best = 0 #INICIALIZANDO O GZ DO MELHOR
for b in best: #O MELHOR É UM DICIONARIO, ONDE CADA CHAVE( OBJETO SPHERE ) GUARDA AS PROPIEDADES DESSA SPHERE( X, Y, M)
    for i in b: #AQUI PEGAMOS APENAS AS CHAVES( SPHERES )
        gz_best += i.Gz(xobs, zobs) #CALCULAMOS O GZ DE TODAS AS SPHERES

In [ ]:
#PLOTANDO A FIGURA
figure, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(17, 12), facecolor='w', sharex=True, squeeze=True)

ax1.plot(xobs, model_gz_noised, 'r-', label='Gz Observado ')
ax1.plot(xobs, gz_best, 'b-', label='Gz Predito ')
ax1.legend()
ax1.set_ylabel('Gz (mGal)', fontsize=15)
ax1.set_xlim(-1000, 1000)
ax1.grid()

ax2.scatter(ga.winner[:, 0], ga.winner[:, 1], color='blue')
ax2.plot([model.params[0], model.params[0], model.params[1], model.params[1], model.params[0]], \
        [model.params[2], model.params[3], model.params[3], model.params[2], model.params[2]], ".-r")

plt.fill_between([model.params[0], model.params[0], model.params[1], model.params[1], model.params[0]], \
                 [model.params[2], model.params[3], model.params[3], model.params[2], model.params[2]], facecolor='red', alpha=0.1)

ax2.scatter(xobs[0:-1:5], zobs[0:-1:5] + 10, s=2)
ax2.set_ylabel('Profundidade (m)', fontsize=15)
ax2.set_ylim(0, 1000)
ax2.set_xlabel('Cordenada x (m)', fontsize=15)
ax2.invert_yaxis()
plt.show( )